In [1]:
#Auteur: Khalid Amghar.

#Exemple du cours: California Mfg

In [2]:
#On charge les modules dont on a besoin
using JuMP    #Pour la définition du modèle (variables, contraintes, fonction objectif, solveur à utiliser...)
using CPLEX   #Pour la résolution du modèle
using LinearAlgebra #Pour pouvoir utiliser des fonctions ou opérateurs de l'algèbre linéaire, comme la fonction
#dot() qui effectue le produit scalaire

In [3]:
#On a 4 installations:
#1. Usine à LA
#2. Usine à SF
#3. Entrepôt à LA
#4. Entrepôt à SF

#Quelles sont les données du problème?
valeurEstimee = [9, 5, 6, 4] #La valeur estimée de: l'usine à LA, l'usine à SF, l'entrepôt à LA et l'entrepôt à SF (respectivement)
coutConst = [6, 3, 5, 2] #Le coût de construction de: l'usine à LA, l'usine à SF, l'entrepôt à LA et l'entrepôt à SF (resp.)
limiteMax = 10 #Le budget maximal alloué à la construction des installations
nbEntrepotsMax = 1 #Le nombre maximal des entrepôts à construire
;

In [4]:
notreModeleBin = Model(with_optimizer(CPLEX.Optimizer)) #On construit un modèle nommé notreModeleBin
#et on indique que le solveur à utiliser est CPLEX

#On ajoute la variable x (un vecteur à 4 composants) au modèle notreModeleBin
@variable(notreModeleBin, x[1:4], Bin ) #On impose que les variables soient binaires (0 ou 1)

@objective(notreModeleBin, Max, dot(valeurEstimee, x))  #Fonction objectif de notreModeleBin

#Les contraintes
@constraint(notreModeleBin, budgetMax, dot(coutConst, x) <= limiteMax)
@constraint(notreModeleBin, limNbEntrepots, x[3] + x[4] <= nbEntrepotsMax)
@constraint(notreModeleBin, contingLA, x[3] <= x[1]) #On ne peut avoir un entrepôt à LA que si l'on a une usine à LA
@constraint(notreModeleBin, contingSF, x[4] <= x[2]) #On ne peut avoir un entrepôt à SF que si l'on a une usine à SF

println(notreModeleBin) #Afficher le modèle
notreModeleBin


Max 9 x[1] + 5 x[2] + 6 x[3] + 4 x[4]
Subject to
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 6 x[1] + 3 x[2] + 5 x[3] + 2 x[4] <= 10.0
 x[3] + x[4] <= 1.0
 x[3] - x[1] <= 0.0
 x[4] - x[2] <= 0.0



A JuMP Model
Maximization problem with:
Variables: 4
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.ZeroOne`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX
Names registered in the model: budgetMax, contingLA, contingSF, limNbEntrepots, x

In [5]:
optimize!(notreModeleBin) #On demande au solveur de résoudre le modèle

Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 4 columns.
MIP Presolve modified 8 coefficients.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)


In [6]:
termination_status(notreModeleBin) #Pour savoir pourquoi l'optimisation a fini 

OPTIMAL::TerminationStatusCode = 1

In [7]:
#Pour les valeurs des variables
println("Solution x: ", value.(x))

Solution x: [1.0, 1.0, 0.0, 0.0]


In [8]:
println("Valeur estimée totale: ", objective_value(notreModeleBin)) #Récupérer la valeur optimale (la valeur de la fonction objectif)

Valeur estimée totale: 14.0


In [9]:
#Pour avoir une solution plus lisible
println("La solution est:")
if value(x[1]) > 0.5
    println("Construire une usine à LA")
end

if value(x[2]) > 0.5
    println("Construire une usine à SF")
end

if value(x[3]) > 0.5
    println("Construire un entrepôt à LA")
end

if value(x[4]) > 0.5
    println("Construire un entrepôt à SF")
end


La solution est:
Construire une usine à LA
Construire une usine à SF
